In [1]:
# System
from dotenv import load_dotenv
import tempfile
from titlecase import titlecase

# LLM Models
from langchain_openai import ChatOpenAI
from openai import OpenAI

# Template
from langchain_core.prompts import ChatPromptTemplate

# OutputParsers
from langchain.schema.output_parser import StrOutputParser

# Gradio frontend
import gradio as gr

In [2]:
load_dotenv()

True

In [3]:
client = OpenAI()

In [4]:
books = {"A Christmas Carol": '.\\data\\A_Christmas_Carol.txt',
                 "A Farewell to Arms": '.\\data\\A_Farewell_to_Arms.txt',
                 "Adventures of Huckleberry Finn": '.\\data\\Adventures_of_Huckleberry_Finn.txt',
                 "Crime and Punishment": '.\\data\\Crime_and_Punishment.txt',
                 "Moby-Dick": '.\\data\\Moby_Dick.txt',
                 "Macbeth": '.\\data\\Macbeth.txt',
                 "Pride and Prejudice": '.\\data\\Pride_and_Prejudice.txt',
                 "Silas Marner": '.\\data\\Silas_Marner.txt',
                 "The Red Badge of Courage": '.\\data\\The_Red_Badge_of_Courage.txt',
                 "Wuthering Heights": '.\\data\\Wuthering_Heights.txt',
                 "Treasure Island": '.\\data\\Treasure_Island.txt'}

In [5]:
def book_headline(book_title="Moby-Dick"):
    filepath = books[book_title]

    with open(file=filepath, mode='r') as file:
        f = file.read()

    f_lines = f.split('\n')
    title = titlecase(f_lines[0])
    author = f_lines[2].title()
    chapters = f.split('[divider]')[1:]
    num_chapters = len(chapters)
    headline = f'{title} {author} \nNumber of Chapters: {num_chapters}'.replace(' By ', ' by ')

    return headline

In [6]:
def book_chapter_summary(book_title="Moby Dick", chapter_num=1):

    filepath = books[book_title]

    with open(file=filepath, mode='r') as file:
        f = file.read()

    f_lines = f.split('\n')
    title = titlecase(f_lines[0])
    author = f_lines[2].title()
    chapters = f.split('[divider]')[1:]

    num_chapters = len(chapters)

    if chapter_num > num_chapters:
        chapter_selected = num_chapters
    else:
        chapter_selected = chapter_num

    chapter = chapters[chapter_selected - 1]

    prompt = ChatPromptTemplate.from_template(
        """
        Provide a summary of the following text, at most 200 words. Answer must be based only on the text given. 
        Context: {chapter}
        """
    )

    chat_model = ChatOpenAI(model="gpt-4.1-nano",
                            max_completion_tokens=256,
                            temperature=0.4)

    chat_chain = prompt | chat_model | StrOutputParser()

    response = chat_chain.invoke({"chapter": chapters[chapter_num - 1]})

    audio = client.audio.speech.create(input=response,
                                       model="gpt-4o-mini-tts",
                                       voice='coral',
                                       instructions="Speak in a story-telling tone.",
                                       response_format="mp3",
                                       speed=1.0)

    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as temp_audio_file:
        temp_audio_file.write(audio.content)
        temp_file_path = temp_audio_file.name

    headline = f'{title} {author} \nChapter {chapter_selected} of {num_chapters}'.replace(' By ', ' by ')

    return headline, response, temp_file_path

In [7]:
def book_chapter_illustration(book_title="Moby Dick", chapter_num=1):
    filepath = book_filepath[book_title]

    with open(file=filepath, mode='r') as file:
        f = file.read()

    f_lines = f.split('\n')
    title = f_lines[0].title()
    author = f_lines[2].title()
    chapters = f.split('[divider]')[1:]

    num_chapters = len(chapters)

    if chapter_num > num_chapters:
        chapter_selected = num_chapters
    else:
        chapter_selected = chapter_num

    chapter = chapters[chapter_selected - 1]

    prompt = ChatPromptTemplate.from_template("""
    Write a caption for a drawing that best capture 
    the essence theme of the following text in 50 words or less. 
    Describe the main characters, their looks, their demeanor, the surrounding, and time period.
    Response must be based only on the text given. 
    context: {chapter}
    """)

    chat_model = ChatOpenAI(model="gpt-4.1-nano",
                                max_completion_tokens=256,
                                temperature=0.4)

    chat_chain = prompt | chat_model | StrOutputParser()

    response = chat_chain.invoke({"chapter": chapters[chapter_num - 1]})
        
    headline = f'{title} {author} \nChapter {chapter_selected} of {num_chapters}'

    return headline, response

In [8]:
css_template = """                   
               h1 {
                   font-size: 36px;
                   font-family: Verdana, Arial, Sans-Serif;
                   font-style: normal;
                   font-weight: bold;
                   # color: black; # rgb(238, 130, 238)
                   # background-color: yellow;
                   text-align: center;
                   text-transform: normal;
                   letter-spacing: 2px;
                   # border: 1px solid lightblue;
                   # border-radius: 0px
                   }

               p {
                   font-size: 14px;
                   font-family: Times New Roman, Times;
                   font-style: normal;
                   font-weight: normal;
                   # color: black; # rgb(238, 130, 238)
                   # background-color: yellow;
                   text-align: left;
                   text-transform: normal;
                   letter-spacing: 0px;
                   # border: 1px solid lightblue;
                   # border-radius: 0px
                   }
               """

In [9]:
with gr.Blocks(theme=gr.themes.Base(),
               title='Book Summary by Chapter',
               css=css_template) as interface:
    gr.Markdown("# Book Summary by Chapter", elem_id='title')
    gr.Markdown("This app summarizes classical novels by chapter. Select the book and the chapter. "
                "Then click on Submit.")
    with gr.Row():  # Create a row to hold two columns
        with gr.Column():  # First column
            dropdown2 = gr.Dropdown(choices=list(books.keys()),
                                    value='Moby-Dick',
                                    multiselect=False,
                                    label="Title of the Book")
            number1 = gr.Number(label="Chapter Number",
                                value=1,
                                minimum=1)
            button1 = gr.Button(value="Submit")
        with gr.Column():  # Second column
            output1 = gr.Text(label="Book Headline")
            output2 = gr.Text(label="Chapter Summary")
            output3 = gr.Audio(label="Generated Audio",
                               type="filepath",
                               autoplay=False,
                               show_share_button=False)
    button1.click(fn=book_chapter_summary,
                  inputs=[dropdown2, number1],
                  outputs=[output1, output2, output3])
    dropdown2.select(fn=book_headline,
                     inputs=[dropdown2],
                     outputs=[output1])

In [10]:
interface.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [11]:
interface.close()

Closing server running on port: 7861
